#### Business Analytics FHDW 2025
## Aufgabe

Mit dem Datensatz *groessen_2.csv* beziehen wir nun auch das Geschlecht mit ein. Führen Sie eine zweifaktorielle Varianzanalyse durch. Formen Sie den DataFrame dazu ggf. passend um.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as sta

groessen_datei_df = pd.read_csv('Daten/groessen_2.csv')

print("Die ursprüngliche Datei:")
print(groessen_datei_df)

groessen_datei_df.index = groessen_datei_df.Geschlecht

# Wir legen eine neue Tabelle mit einer anderen Struktur an:
groessen_df = pd.DataFrame(columns=['Sportart', 'w', 'm'])
groessen_df.index = groessen_df.Sportart

print("Alle Spalten außer Geschlecht liefern die Sportarten:")
print(groessen_datei_df.columns.difference(['Geschlecht']))

for c in groessen_datei_df.columns.difference(['Geschlecht']):
    # Wir bauen direkt für jede Sportart c
    # die kompletten Spalten für w und m:
    col_w = groessen_datei_df[c].loc['w'].values
    col_m = groessen_datei_df[c].loc['m'].values
    # Dann iterieren wir über die Spalten und fügen mit deren Inhalten
    # einen neuen Datensatz an unsere neue Tabelle:
    for i in range(0, len(col_w)):
        groessen_df = pd.concat([groessen_df, 
                                 pd.DataFrame([{'Sportart':c, 'w':col_w[i], 'm':col_m[i]}])], ignore_index=True)

groessen_df

In [ ]:
groessen_df.index=groessen_df.Sportart

mittelwerte_df = pd.DataFrame(columns=['w','m','Randmittel', 'Wahrer Effekt'], 
                              index=['Turnen','Fussball','Volleyball','Randmittel','Wahrer Effekt'])
for p in set(groessen_df.index):
        mittelwerte_df['w'][p] = groessen_df['w'][p].mean()
        mittelwerte_df['m'][p] = groessen_df['m'][p].mean()
        mittelwerte_df['Randmittel'][p] = (mittelwerte_df['w'][p]+mittelwerte_df['m'][p])/2

mittelwerte_df['w']['Randmittel'] = mittelwerte_df['w'].mean()
mittelwerte_df['m']['Randmittel'] = mittelwerte_df['m'].mean()
mittelwerte_df['Randmittel']['Randmittel'] = (mittelwerte_df['w']['Randmittel']
                                                          +mittelwerte_df['m']['Randmittel'])/2

mittelwerte_df['Wahrer Effekt'] = mittelwerte_df['Randmittel']-mittelwerte_df['Randmittel']['Randmittel']
mittelwerte_df['w']['Wahrer Effekt'] = mittelwerte_df['w']['Randmittel']-mittelwerte_df['Randmittel']['Randmittel']
mittelwerte_df['m']['Wahrer Effekt'] = mittelwerte_df['m']['Randmittel']-mittelwerte_df['Randmittel']['Randmittel']

mittelwerte_df

Grafische Darstellung der Faktorstufenmittelwerte in Abhängigkeit zu den Stufen eines anderen Faktors. Interaktionswirkung zeigt sich hier, wenn die Verbindungslinien zwischen den Mittelwerten *nicht parallel* verlaufen.

In [ ]:
plt.xticks(ticks=[0,1,2], labels=['Turnen','Fussball','Volleyball'])
plt.plot([mittelwerte_df.w.Turnen, 
          mittelwerte_df.w.Fussball, 
          mittelwerte_df.w.Volleyball], label='w')
plt.plot([mittelwerte_df.m.Turnen, 
          mittelwerte_df.m.Fussball, 
          mittelwerte_df.m.Volleyball], label='m')
plt.legend()
plt.show()

### Zerlegung der Streuung und Modellgüte

In [ ]:
G = 3 # Anzahl Sportarten (Faktorstufen Faktor A)
H = 2 # Anzahl Geschlechter (Faktorstufen Faktor B)
N = 5 # Anzahl Elemente in Zelle (g, h)
gesamtmittel = mittelwerte_df.Randmittel.Randmittel

faktor_A = ['Turnen','Fussball','Volleyball']
faktor_B = ['w', 'm']

SSA = H * N * np.sum([(mittelwerte_df.Randmittel[g]-gesamtmittel)**2 for g in faktor_A])
SSB = G * N * np.sum([(mittelwerte_df[h].Randmittel-gesamtmittel)**2 for h in faktor_B])
SSAxB = N * np.sum([np.sum([(mittelwerte_df[h][g]
                             -(mittelwerte_df[h].Randmittel+mittelwerte_df.Randmittel[g]-gesamtmittel))**2 
                            for h in faktor_B]) 
                    for g in faktor_A])
SSw = np.sum([np.sum([np.sum([(groessen_df[h][g][i]-mittelwerte_df[h][g])**2 for i in range(0,N)]) for h in faktor_B]) for g in faktor_A])
SSb = SSA + SSB + SSAxB
SSt = SSb + SSw

$EtaQuadrat_{faktor} = \frac{SS_{faktor}}{SS_{faktor}+SS_w}$ als eine erste Beurteilung der Modellgüte:

In [ ]:
eta_quadrat = SSb/SSt
print("Eta-Quadrat = {:.4f}%, {:.4f}% bleiben unerklärt.".format(eta_quadrat*100,(1-eta_quadrat)*100))
partielle_eta_Q_df = pd.DataFrame({'Quelle': ['Sportart', 'Geschlecht', 'Interaktion'],
                                   'Quadrate (erklärt)': [SSA, SSB, SSAxB],
                                   'Fehler (unerklärt)': [SSw, SSw, SSw],
                                   'Gesamtabweichung': [SSA+SSw, SSB+SSw, SSAxB+SSw],
                                   'Partielles Eta-Quadrat': [SSA/(SSA+SSw),SSB/(SSB+SSw),SSAxB/(SSAxB+SSw)]})
partielle_eta_Q_df

### Prüfung der statistischen Signifikanzen

Der Signifikanztest für das zweifaktorielle Modell entspricht als Quotient aus erklärter und nicht erklärter Varianz dem einfachen Modell, berücksichtigt aber unterschiedliche Freiheitsgrade:

$F_{emp} = \frac{SS_b/(G \cdot H-1)}{SS_w/(G \cdot H \cdot (N - 1))}=\frac{MS_b}{MS_w}$

Dazu setzen wir wieder die *ANOVA-Tabelle* zusammen.

In [ ]:
df_A = G-1
df_B = H-1
df_AxB = (G-1)*(H-1)
df_w = G*H*(N-1)
df_t = G*H*N-1
df_b = G*H-1

MSb = SSb/df_b
MSw = SSw/df_w
F_emp = MSb/MSw

anova_df = pd.DataFrame({'Varianzquelle': ['Haupteffekte', 'Sportart', 'Geschlecht', 'Interaktion', 'Sportart*Geschlecht', 'Reststreuung', 'Total'],
                         'SS': ['-', SSA, SSB, '-', SSAxB, SSw, SSt],
                         'df': ['-', df_A, df_B, '-', df_AxB, df_w, df_t],
                         'MS': ['-', SSA/df_A, SSB/df_B, '-', SSAxB/df_AxB, SSw/df_w, SSt/df_t]})
anova_df

Führen wir zunächst den zusammengefassten bzw. globalen Test durch.

Für $\alpha=0.05$ ermitteln wir den passenden Wert der F-Verteilung $F_\alpha$, den wir mit dem $F_{emp}$ von oben vergleichen:

In [ ]:
F_alpha = sta.f.ppf(0.95, dfn=df_b, dfd=df_w)
p_value = (1 - sta.f.cdf(F_emp, dfn=df_b, dfd=df_w))
print('F_emp = {:.2f}\nF_alpha = {:.2f}\np = {}'.format(F_emp, F_alpha, p_value))

Damit gilt wieder $F_{emp} > F_\alpha$ und wir verwerfen hier die Nullhypothese: Wir können also davon ausgehen, dass mindestens eine der Faktorstufen Einfluss auf die Größe hat. Der dazugehörige $p$-Wert ist praktisch null.

Für das Konfidenzniveau von 95% betrachten wir nun auch noch die spezifischen F-Tests:

In [ ]:
spez_F_tests = pd.DataFrame({'Varianzquelle': ['Sportart','Geschlecht','Interaktion'],
                             'df (Zähler)': [df_A, df_B, df_AxB],
                             'df (Nenner)': [df_w, df_w, df_w],
                             'F_alpha': [sta.f.ppf(0.95, dfn=df_A, dfd=df_w), 
                                         sta.f.ppf(0.95, dfn=df_B, dfd=df_w), 
                                         sta.f.ppf(0.95, dfn=df_AxB, dfd=df_w)],
                             'F_emp': [(SSA/df_A)/MSw, (SSB/df_B)/MSw, (SSAxB/df_AxB)/MSw]})
spez_F_tests

Hier sehen wir, dass für die beiden Faktoren die Nullhypothese verworfen werden kann, für die Interaktion aber nicht. Sportart und Geschlecht haben isoliert betrachtet jeweils eine Wirkung auf die Größe, die *gemeinsame Wirkung* von Sportart und Geschlecht ist aber nicht signifikant.